In [51]:
# IMPORTS

import numpy as np
from matplotlib import pyplot as plt
from pprint import pprint

## Exercice 1

### 1) Generate n=500 samples from a Beta distribution with parameter (α,β)=(2,5). Display the histogram of this sample with 25 bins. Then, generate n=500 independent random vectors Xi , i=1,.,n in R2, where all coordinates are drawn independently from a Beta distribution with parameter (α,β)=(2,5). Compute the mean vector (in R²).

In [63]:
(alpha, beta) = (2, 5)
n = 500

samples = np.random.beta(alpha, beta, n)
plt.hist(samples, bins=25, label='Samples from Beta distribution')
plt.show()

beta1 = np.random.beta(alpha, beta, 500)
beta2 = np.random.beta(alpha, beta, 500)
r2_vectors = list(zip(beta1, beta2))

mean_vector = (beta1.mean(), beta2.mean())
mean_vector

(0.27857926087005253, 0.28496940343875404)

### 2) Compute B=500 bootstrap estimators of the mean. On the same plot, represent the observed data, the mean and the 500 bootstrap estimators of the mean.


In [67]:
B = 500

bootstrap_samples = []
print(np.random.randint(0, n, n))
# for _ in range(B):
#     bootstrap_samples.append(r2_vectors[list(np.random.randint(0, n, n))])

bootstrap_samples_mean = [v.mean() for v in bootstrap_samples]
# mean_vect = []

# mean_vector = vectors.mean(axis=0)
# len(mean_vector)
# mean_vector
# plt.hist(mean_vector, bins=25, label='Samples from Beta distribution')
# plt.show()

[336 257 180 275 163 320 149 425 210   2 271 140 104 217 136 430 174 332
  68 156 381 493 363 195  65 321 489 284  90  20 119 463 306 410 490 191
 219 172  88 211 252 192 383 218 375 105  12  94 359 241 375 303 400  23
 153 302 374  12  80 108 155 107 379  29 107 211 148 287 108 106  80 323
 283 244 320 479  41 353  56  91 113 491 136 364 137 275 234 314 336 123
 163 171 381 136 259 155  94 182 262 207 467 212 241  12 469 290 259 266
 159 328 255  91 144 353 205 202 110 323  15 369 481 197 193   0 427 450
 416 336 216  18 234 175 182 194  11 227 388  83  15  86   3  81 489 194
 472 361 477 174 128  45 310  80 287  26 440 430 193 286  17 212 460 377
 133 160 321 413 177  74 399 208 277  59 178 158 395 110 330 367 156 492
 324 103 478 153 357 338 254 163 458 312  97 378 195 492 188 313 185 348
  33 127 212 158 408  36 199   5 454 361   6 272 136 272  91 196 440 353
 128  15 274 299 436 138 302 265  32 157 436 453  19 496  96 309 415 140
 193 429 454 177 229 255 292  91 158 351 255 444 48